In [ ]:
from collections.abc import Generator, Callable
from pathlib import Path
import typing
from typing import Any, TypeAlias
import pandas as pd
import numpy as np
import datetime as dt
import re
from functools import partial, reduce
from tqdm import tqdm
from IPython.display import (
    display, # type: ignore[reportUnknownVariableType]
    Markdown,
)

import importlib

from config.fastf1 import fastf1
from config import config
import src.data.constants as dataset_constants
importlib.reload(dataset_constants);
import src.data.loader
importlib.reload(src.data.loader);
from src.data.loader import stream_ndjson, load_submissions_df, load_comments_df
from src.data.preprocessing import concatenate_submissions_and_comments

from src.utils import (
    temporary_pandas_options,
    display_full_dataframe,
    hide_index,
    compose,
)
from src import utils
utils.set_random_seeds()
DEVICE = utils.get_device()

import logging
logging.getLogger('fastf1').setLevel(logging.WARNING)

# Load data and find submissions related to steward decisions

In [53]:
f1_submissions_df = load_submissions_df(
    dataset_constants.RawFile.FORMULA1_SUBMISSIONS,
    columns=dataset_constants.DEFAULT_SUBMISSION_COLUMNS | {'permalink', 'post_hint', 'link_flair_text'},
)                                  

f1_comments_df = load_comments_df(
    dataset_constants.RawFile.FORMULA1_COMMENTS,
    columns=dataset_constants.DEFAULT_COMMENT_COLUMNS | {'link_id'},
)

In [54]:
f1_comments_df = f1_comments_df[~f1_comments_df['body'].isin({'[removed]', '[deleted]'})]

In [3]:
# TODO: testing purposes
f1_submissions_df['permalink'] = 'www.reddit.com' + f1_submissions_df['permalink']

In [ ]:
steward_decision_related_words = {
    'penalty', 'steward', 'decision', 'appeal', 'review', 'ruling', 'investigation', 'regulation',
    'seconds', 'sec', 
    'collision', 'crash', 'incident', 'overtake', 'virtual safety car', 'blocking', 'brake test', 'contact',
    'red flag', 'yellow flag', 
    'controversial', 'rigged', 'corrupt', 'bias', 'protest', 'FIA', 'document', 'infringement'}

# Manually exclude some posts unrelated to steward decisions
excluded_submission_ids = {
    'vdr1c6',
    'w7z5aj',
    'wf87e0',
    'x1zd5z',
    'x3y140',
}

words_regex = ''.join(fr'\b{word}\b|' for word in steward_decision_related_words)[:-1]
steward_decision_pattern = re.compile(words_regex, flags=re.IGNORECASE)

relevant_flairs = {':post-technical: Technical', ':post-news: News'}

has_related_words = f1_submissions_df['title'].apply(lambda title: steward_decision_pattern.search(title) is not None)
has_relevant_flairs = f1_submissions_df['link_flair_text'].isin(relevant_flairs)
is_image_post = f1_submissions_df['post_hint'] == 'image'
is_included = ~f1_submissions_df['id'].isin(excluded_submission_ids) 

steward_decision_submissions_df = f1_submissions_df[has_related_words & has_relevant_flairs & is_image_post & is_included].copy()

with display_full_dataframe():
    print(len(steward_decision_submissions_df))
    display(steward_decision_submissions_df.head(2))

In [ ]:
submission_link_id = f't3_xtqa50'
comments_df = f1_comments_df[f1_comments_df['link_id'] == submission_link_id]
print(len(comments_df))


# Discretization of continuous sentiment function

In [6]:
def to_sentiment_category(sentiment: float) -> str:
    if sentiment >= 0.05:
        return 'Positive'
    elif sentiment <= -0.05:
        return 'Negative'
    else:
        return 'Neutral'

### VADER SENTIMENT ANALYSIS

In [7]:
# TODO: testing
steward_decision_submissions_df = load_submissions_df(dataset_constants.RawFile.FORMULA1_SUBMISSIONS, partial(stream_ndjson, limit=100))

In [8]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()

In [ ]:
individual_vader_sentiments = []

for index, steward_decision_submission in steward_decision_submissions_df.iterrows():
    submission_link_id = f't3_{steward_decision_submission['id']}'
    comments_df = f1_comments_df[f1_comments_df['link_id'] == submission_link_id].copy()

    if comments_df.empty:
        steward_decision_submissions_df.loc[index, 'average_sentiment_vader'] = np.nan
        continue
    
    NOEMER = np.abs(comments_df['score']).sum()
    
    if NOEMER == 0:
        steward_decision_submissions_df.loc[index, 'average_sentiment_vader'] = np.nan
        continue

    comments_df.loc[:, 'compound'] = comments_df['body'].apply(
        lambda text: analyzer.polarity_scores(text)['compound']
    )   #creates a data series with a score (float) given an input comment

    sentiment = (comments_df['compound'] * comments_df['score']).sum() / NOEMER
    steward_decision_submissions_df.loc[index, 'average_sentiment_vader'] = sentiment

    individual_vader_sentiments.append(sentiment)

# print(individual_vader_sentiments[:10])

with display_full_dataframe():
    display(steward_decision_submissions_df.head(2))



In [ ]:
# # TOOD: vectorize if efficiency is needed
# steward_decision_submissions_df['submission_link_id'] = 't3_' + steward_decision_submissions_df['id']

# filtered_comments_df = f1_comments_df[
#     f1_comments_df['link_id'].isin(steward_decision_submissions_df['submission_link_id'])
# ].copy()

# filtered_comments_df['compound'] = filtered_comments_df['body'].apply(
#     lambda text: analyzer.polarity_scores(text)['compound']
# )

# def calculate_weighted_sentiment(group):
#     NOEMER = np.abs(group['score']).sum()
#     if NOEMER == 0:
#         return np.nan
#     return (group['compound'] * group['score']).sum() / NOEMER

# average_sentiment = filtered_comments_df.groupby('link_id').apply(calculate_weighted_sentiment)

# steward_decision_submissions_df['average_sentiment_vader'] = \
#     steward_decision_submissions_df['submission_link_id'].map(average_sentiment)

# steward_decision_submissions_df.drop(columns=['submission_link_id'], inplace=True)


### BERT SENTIMENT ANALYSIS

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("cardiffnlp/twitter-roberta-base-sentiment-latest")
model = AutoModelForSequenceClassification.from_pretrained("cardiffnlp/twitter-roberta-base-sentiment-latest")
model.to(DEVICE);

In [38]:
import torch
from transformers import RobertaTokenizer, RobertaForSequenceClassification

def BERT_sentiment(text):
    inputs = tokenizer(text, return_tensors='pt', truncation=True, padding=True, max_length=512)
    with torch.no_grad():
        outputs = model(**inputs)
    logits = outputs.logits
    sentiment_score = torch.argmax(logits, dim=1).item()  # 0: negative, 1: neutral, 2: positive
    return (sentiment_score - 1) #-1: negative, 0: neutral, 1: positive

individual_bert_sentiments = []

for index, steward_decision_submission in steward_decision_submissions_df.iterrows():
    submission_link_id = f't3_{steward_decision_submission['id']}'
    comments_df = f1_comments_df[f1_comments_df['link_id'] == submission_link_id].copy()

    if comments_df.empty:
        steward_decision_submissions_df.loc[index, 'average_sentiment_bert'] = np.nan
        individual_bert_sentiments.append(sentiment)  # Store NaN in the list
        continue
    
    NOEMER = np.abs(comments_df['score']).sum()
    
    if NOEMER == 0:
        steward_decision_submissions_df.loc[index, 'average_sentiment_bert'] = np.nan
        individual_bert_sentiments.append(sentiment)  # Store NaN in the list
        continue

    comments_df.loc[:, 'compound'] = comments_df['body'].apply(BERT_sentiment)
        # lambda text: analyzer.polarity_scores(text)['compound']
        # bert induced sentiment

    sentiment = (comments_df['compound'] * comments_df['score']).sum() / NOEMER
    steward_decision_submissions_df.loc[index, 'average_sentiment_bert'] = sentiment

    individual_bert_sentiments.append(sentiment)  # Append BERT sentiment to list

#  with display_full_dataframe():
    # display(steward_decision_submissions_df)
    # display(steward_decision_submissions_df['average_sentiment_bert'])

In [ ]:
with display_full_dataframe():
    display(steward_decision_submissions_df)

In [ ]:
# MEAN ABSOLUTE ERROR
print(
    np.abs(steward_decision_submissions_df['average_sentiment_bert'] - steward_decision_submissions_df['average_sentiment_vader']).sum() \
    / len(steward_decision_submissions_df.index)
)

In [ ]:
display(steward_decision_submissions_df.columns)


# VALIDATION
# 1. CATEGORIZATIONI AGREEMENT METRIC - Cohen's Kappa

In [ ]:
from sklearn.metrics import cohen_kappa_score

# Vader_sentiments = steward_decision_submissions_df['average_sentiment_vader']
# Bert_sentiments = steward_decision_submissions_df['average_sentiment_bert']

vader_sentiment_labels = np.array([to_sentiment_category(x) for x in individual_vader_sentiments])
bert_sentiment_labels = np.array([to_sentiment_category(y) for y in individual_bert_sentiments])

COHEN_KAPPA = cohen_kappa_score(vader_sentiment_labels, bert_sentiment_labels)
print(f"Cohen's Kappa: {COHEN_KAPPA:.2f}")

# Kappa > 0.75 is a Strong agreement
# Kappa = 0.4 - 0.75 is a Moderate agreement
# Kappa < 0.4 is a Weak agreement


# 2. BIAS DETECTION METRIC - Bland-Altman plot

In [ ]:
# calculate systematic biases between VADER and BERT sentiment analysis

import seaborn as sns
import matplotlib.pyplot as plt


mean_scores = (np.array(individual_vader_sentiments) + np.array(individual_bert_sentiments)) / 2
diff_scores = np.array(individual_vader_sentiments) - np.array(individual_bert_sentiments)

# Create a scatter plot for the Bland-Altman analysis
sns.scatterplot(x=mean_scores, y=diff_scores)
plt.axhline(0, color='red', linestyle='dashed')  # No bias line
plt.xlabel("Mean Sentiment Score")
plt.ylabel("VADER - BERT Sentiment Score Difference")
plt.title("Bland-Altman Plot")
plt.show()
# patterns indicate bias
# if most points are close to 0, the models mostly agree